In [19]:
import pandas as pd
from real_estate.finca_raiz import main
import h3
from shapely import wkt
import geopandas as gpd
from shapely import Polygon
import numpy as np

In [35]:
def get_geojson_border(h3_id):
    if h3.h3_is_valid(h3_id):
        return Polygon(h3.h3_to_geo_boundary(h3_id, geo_json=True))
    

In [39]:
def get_h3_borders(h3_series: pd.Series) -> gpd.GeoSeries:
    borders = h3_series.apply(get_geojson_border)
    return gpd.GeoSeries(borders)

In [3]:
sc = main.FincaRaizClient(20)

In [4]:
test = sc.search(offer="sell", property_type="apartment")

In [19]:
import plotly.graph_objects as go

mapper = {
    "rooms.name": "No. Habitaciones",
    "baths.name": "No. Baños",
    "stratum.name": "Estrato",
}

column = "rooms.name"

grouped_data = (
    test
    .groupby(column, as_index=False)
    .size()
    .sort_values(column)
)
fig = go.Figure(
    data=[go.Pie(labels=grouped_data[column], values=grouped_data["size"], hole=.5, textinfo='label+percent', sort=False, direction="clockwise")]
)
fig.update_layout(
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=False,
    title=dict(
        text=mapper[column],
        x=0.5,
        xanchor="center",
    ),
    height=300,
)

In [9]:
test["geometry"] = test["locations.location_point"].apply(wkt.loads)
test["lat"] = test["geometry"].apply(lambda p: p.y)
test["lon"] = test["geometry"].apply(lambda p: p.x)
test["lat"] = test["lat"].where(test["lat"].between(4, 11), pd.NA)
test["lon"] = test["lon"].where(test["lon"].between(-76, -73), pd.NA)

In [115]:
test["h3"] = test.apply(lambda x:
    h3.geo_to_h3(x["lat"], x["lon"], 4), axis=1
)

In [116]:
count_hex = test.groupby("h3", as_index=False).size()

In [117]:
count_hex["h3_geometry"] = get_h3_borders(count_hex["h3"])
count_hex = count_hex.dropna()


In [118]:
from geojson import Feature, FeatureCollection
def gdf_to_json(
    gdf: gpd.GeoDataFrame,
    id_col: str,
    geometry_col: str,
    value_field: str,
) -> FeatureCollection:
    """Return geojson from GeoDataFrame."""
    list_features = []
    for _, row in gdf.iterrows():
        feature = Feature(
            geometry=row[geometry_col],
            id=row[id_col],
            properties={"value": row[value_field]},
        )
        list_features.append(feature)
    feat_collection = FeatureCollection(list_features)
    return feat_collection

In [119]:
import plotly.express as px

In [120]:
from dotenv import load_dotenv
import os
load_dotenv("../eterna-primavera/.env")

True

In [121]:
geojson_obj = gdf_to_json(count_hex, "h3", "h3_geometry", "size")

In [122]:
import json
with open("test.geojson", "w") as f:
    json.dump(geojson_obj, f)

In [123]:

fig = px.choropleth_mapbox(
        count_hex,
        geojson=geojson_obj,
        locations="h3",
        color="size",
        mapbox_style="carto-positron",
        zoom=9,
        center={"lat": 6.240833, "lon": -75.530553},
        # TODO: Add extra info and show get_name() on hover
        # hover_data={
        #     "size": True,
        #     "h3": False
        # }
    )
    
fig = fig.update_layout(
    mapbox_style=os.environ["MAPBOX_STYLE"],
    mapbox_accesstoken=os.environ["MAPBOX_TOKEN"],
    margin=dict(l=0, r=0, t=0, b=0),
    uirevision="Don't change",  # this option forces zoom to stay the same when data is updated),
    showlegend=False,
)
fig